In [29]:
import tpqoa
from iterativeBase import *
from scipy.optimize import brute
from scipy.optimize import minimize

In [60]:
class IterativeBacktest(IterativeBase):
    ''' Class for iterative (event-driven) backtesting of trading strategies.
    '''

    # helper method
    def go_long(self, bar, units = None, amount = None):
        if self.position == -1:
            self.buy_instrument(bar, units = -self.units) # if short position, go neutral first
        if units:
            self.buy_instrument(bar, units = units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.buy_instrument(bar, amount = amount) # go long

    # helper method
    def go_short(self, bar, units = None, amount = None):
        if self.position == 1:
            self.sell_instrument(bar, units = self.units) # if long position, go neutral first
        if units:
            self.sell_instrument(bar, units = units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.sell_instrument(bar, amount = amount) # go short

    def test_sma_strategy(self, SMA_S, SMA_L):
        
        # nice printout
        stm = "Testing SMA strategy | {} | SMA_S = {} & SMA_L = {}".format(self.symbol, SMA_S, SMA_L)
        lines = "-" * 75
        print(lines)
        print(stm)
        print(lines)
        
        # reset 
        self.position = 0  # initial neutral position
        self.trades = 0  # no trades yet
        self.current_balance = self.initial_balance  # reset initial capital
        self.report[:] = [lines, stm, lines]
        self.get_data() # reset dataset
        
        # prepare data
        self.data["SMA_S"] = self.data["price"].rolling(SMA_S).mean()
        self.data["SMA_L"] = self.data["price"].rolling(SMA_L).mean()
        self.data.dropna(inplace = True)

        bar = 0
        
        # sma crossover strategy
        for bar in range(len(self.data)-1): # all bars (except the last bar)
            if self.data["SMA_S"].iloc[bar] > self.data["SMA_L"].iloc[bar]: # signal to go long
                if self.position in [0, -1]:
                    self.go_long(bar, amount = "all") # go long with full amount
                    self.position = 1  # long position
            elif self.data["SMA_S"].iloc[bar] < self.data["SMA_L"].iloc[bar]: # signal to go short
                if self.position in [0, 1]:
                    self.go_short(bar, amount = "all") # go short with full amount
                    self.position = -1 # short position
            self.data.loc[self.data.index[bar], "nav"] = self.current_balance + self.units * self.data["price"].iloc[bar]
            
        self.close_pos(bar+1) # close position at the last bar
        self.data_clean_up(bar+1)
        return self.current_balance 

                
    def test_boll_strategy(self, SMA, dev):

        # nice printout
        stm = "Testing Bollinger Bands Strategy | {} | SMA = {} & dev = {}".format(self.symbol, SMA, dev)
        lines = "-" * 75
        print(lines)
        print(stm)
        print(lines)
        
        # reset 
        self.position = 0  # initial neutral position
        self.trades = 0  # no trades yet
        self.current_balance = self.initial_balance  # reset initial capital
        self.report[:] = [lines, stm, lines]
        self.get_data() # reset dataset
        
        # prepare data
        self.data["SMA"] = self.data["price"].rolling(SMA).mean()
        self.data["Lower"] = self.data["SMA"] - self.data["price"].rolling(SMA).std() * dev
        self.data["Upper"] = self.data["SMA"] + self.data["price"].rolling(SMA).std() * dev
        self.data.dropna(inplace = True) 
        
        # Bollinger strategy
        for bar in range(len(self.data)-1): # all bars (except the last bar)
            if self.position == 0: # when neutral
                if self.data["price"].iloc[bar] < self.data["Lower"].iloc[bar]: # signal to go long
                    self.go_long(bar, amount = "all") # go long with full amount
                    self.position = 1  # long position
                elif self.data["price"].iloc[bar] > self.data["Upper"].iloc[bar]: # signal to go Short
                    self.go_short(bar, amount = "all") # go short with full amount
                    self.position = -1 # short position
            elif self.position == 1: # when long
                if self.data["price"].iloc[bar] > self.data["SMA"].iloc[bar]:
                    if self.data["price"].iloc[bar] > self.data["Upper"].iloc[bar]: # signal to go short
                        self.go_short(bar, amount = "all") # go short with full amount
                        self.position = -1 # short position
                    else:
                        self.sell_instrument(bar, units = self.units) # go neutral
                        self.position = 0
            elif self.position == -1: # when short
                if self.data["price"].iloc[bar] < self.data["SMA"].iloc[bar]:
                    if self.data["price"].iloc[bar] < self.data["Lower"].iloc[bar]: # signal to go long
                        self.go_long(bar, amount = "all") # go long with full amount
                        self.position = 1 # long position
                    else:
                        self.buy_instrument(bar, units = -self.units) # go neutral
                        self.position = 0    
            self.data.loc[self.data.index[bar], "nav"] = self.current_balance + self.units * self.data["price"].iloc[bar]

        self.close_pos(bar+1) # close position at the last bar
        self.data_clean_up(bar+1)
        return self.current_balance 
    
    def test_macd_strategy(self, EMA_S, EMA_L, signal):
        
        # nice printout
        stm = "Testing MACD strategy | {} | EMA_S = {}, EMA_L = {}, & Signal = {}".format(self.symbol, EMA_S, EMA_L, signal)
        lines = "-" * 75
        print(lines)
        print(stm)
        print(lines)
        
        # reset 
        self.position = 0  # initial neutral position
        self.trades = 0  # no trades yet
        self.current_balance = self.initial_balance  # reset initial capital
        self.report[:] = [lines, stm, lines]
        self.get_data() # reset dataset
        
        # prepare data
        ema_s = self.data["price"].ewm(span=EMA_S, min_periods=EMA_S, adjust=False).mean()
        ema_l = self.data["price"].ewm(span=EMA_L, min_periods=EMA_L, adjust=False).mean()
        self.data["macd"] = ema_s - ema_l

        self.data["signal"] = self.data["macd"].ewm(span=signal, min_periods=signal, adjust=False).mean()
        self.data.dropna(inplace = True)

        self.data["nav"] = 0
        for bar in range(len(self.data)-1): # all bars (except the last bar)
            if self.data["signal"].iloc[bar] > self.data["macd"].iloc[bar]: # signal to go long
                if self.position in [0,-1]:
                    self.go_long(bar, amount = "all") # go long with full amount
                    self.position = 1  # long position

            elif self.data["signal"].iloc[bar] < self.data["macd"].iloc[bar]: # signal to go short
                if self.position in [0,1]:
                    self.go_short(bar, amount = "all") # go short with full amount
                    self.position = -1 # short position

            self.data.loc[self.data.index[bar], "nav"] = self.current_balance + self.units * self.data["price"].iloc[bar]

        self.close_pos(bar+1) # close position at the last bar
        self.data_clean_up(bar+1)
        return self.current_balance 

    def data_clean_up(self, bar):
        self.data.loc[self.data.index[bar], "nav"] = self.current_balance 

        self.data["strategy"] = np.log(self.data.nav / self.data.nav.shift(1))
        self.data["creturns"] = self.data["returns"].cumsum().apply(np.exp)
        self.data["cstrategy"] = self.data["strategy"].cumsum().apply(np.exp)

    def optimise_parameters(self, func, *ranges, early_termination_threshold=None):
        best_value = float('-inf') 
        best_combination = None

        def generate_combinations():
            for r in ranges:
                yield np.arange(r[0], r[1] + 1)

        for combo in itertools.product(*generate_combinations()):
            value = func(*combo)
            
            if early_termination_threshold is not None and value >= early_termination_threshold:
                return combo, value
            
            if value > best_value:
                best_value = value
                best_combination = combo

        return best_combination, best_value

    def plot_performance(self):
        # Change the style of plot
        plt.style.use('seaborn-darkgrid')
        
        # Create a color palette
        palette = plt.get_cmap('Set1')
        
        # Plot multiple lines

        plt.plot(self.data.index, self.data["creturns"], marker='', color=palette(1), linewidth=1, alpha=0.9, label="Cumulative Returns")
        plt.plot(self.data.index, self.data["cstrategy"], marker='', color=palette(2), linewidth=1, alpha=0.9, label="Cumulative Strategy Returns")

        # Add legend
        plt.legend(loc=2, ncol=2)
        
        # Add titles
        plt.title("Performance of Strategy against buy and hold over time", loc='left', fontsize=12, fontweight=0, color='orange')
        plt.xlabel("Time")
        plt.ylabel("Performance")

        # Show the graph
        plt.show()


In [61]:
test = IterativeBacktest("/Users/dugaldmacintyre/Desktop/Oanda_firststeps/oanda.cfg", "EUR_USD", "2010-01-01", "2020-12-31", "D", 100000, use_spread=True)